

<br>
<h1><TT>It's Officially Legal so Let's Scrape the Web</TT></h1>
<br>
Kimberly Fessel  

- Twitter @kimberlyfessel 
- LinkedIn kimberlyfessel
<br> 

<h2> <TT> Scraping Pipeline with Wikipedia </TT> </h2>

<br>

---

# Getting Started

We will now be parsing information collected from the Internet, specifically from Wikipedia.  

First let's take a look at the HTML source code that powers the page about Pennsylvania:
- Open up https://en.wikipedia.org/wiki/Pennsylvania in your browser
- Right click and select "Inspect" or "Inspect Element"
- Alternatively:
  - _Chrome_ -- View > Developer > View Source
  - _Safari_ -- Develop > Show Web Inspector 
  - _Firefox_ -- Tools > Web Developer > Inspector

The same HTML code we have been exploring is used to produce the structure of just about every webpage you visit.
<br><br>

Note: we will be scraping Wikipedia for learning purposes, but you can simply download its content instead.  Check [this](https://en.wikipedia.org/wiki/Wikipedia:Database_download) out to learn more.

### Introduction to `requests`

The Python library `requests` allows us to retrieve information from the web.  

First we need to import this package.

In [ ]:
import requests

Now use the `.get()` method to retrieve a page's HTML.

In [ ]:
url = 'https://en.wikipedia.org/wiki/Pennsylvania'

response = requests.get(url)

So, what's in a response?

This object gives us a few important things:
- `response.text` -- the returned HTML (if any)
- `response.status_code` -- a [code](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes) to tell you if your request was successful

In [ ]:
response.status_code    #200 = success

In [ ]:
print(response.text[:200])   #First 200 characters of the HTML

In [ ]:
page = response.text

In [ ]:
type(page)

### Using `requests` with `BeautifulSoup`

Now that we have the HTML, we use `BeautifulSoup` to understand its structure in the exact same way as we did with sample HTML.

<br>
<center>
<img src="https://drive.google.com/uc?export=view&id=1OVCQBlNyA8RTkilbsMfEHP_7j000G1DP"  alt="info" height="100"/>
</center>

In [ ]:
from bs4 import BeautifulSoup as bs

In [ ]:
soup = bs(page)

`BeautifulSoup` has now parsed through the HTML about Pennsylvania, so we can look for things like the header tag:

In [ ]:
soup.find('h1')

In [ ]:
soup.find('h1').text

To extract information from the web, you will alternate between: 
- Inspecting the HTML in your browser
- Using `BeautifulSoup` to find information

### Exercises

**Class Exercise - Disambiguation link**  <br> <br>
Wikipedia often provides a disambiguation link to a list of additional topics that could reference the same term.  For example, searching for "Pennsylvania" directs to this article about the state, but "Pennsylvania" may instead refer to the railroad, a ship, or a music album.

>Let's try to extract this disambiguation link by inspecting the source code and then using `BeautifulSoup`.  

**Exercise 1 - Longitute, Latitude**  <br>
>How would you retrieve Pennsylvania's longitude and latitude coordinates from this page?

<center>
<img src="https://drive.google.com/uc?export=view&id=1N43WC3jwFpV80L7-iLnEZi8aCQZtWcd3"  alt="coordinates" width="600"/>
</center>

_Hint: Right click the coordinates directly and then select inspect._

# Advancing Further

### Chaining commands

In the last notebook, we learned that we could first isolate a division of the HTML and then look for tags within the division.  

The returned element(s) of any `find()` or `find_all()` command are themselves `BeautifulSoup` elements.  This means we can continue searching for information within them.

Let's take a look at the first table on the page.

_Note: Adding `.prettify()` below just prints each HTML element on its own line._

In [ ]:
print(soup.find('table').prettify())

In [ ]:
first_table = soup.find('table')

In [ ]:
type(first_table)

Now find the header text of this table and the text of the first data row.

In [ ]:
first_table.find('th').text

In [ ]:
first_table.find('td').text

Also note that instead of saving the table as its own Python variable, you could just chain these searches together.

In [ ]:
state = soup.find('table').find('th').text

state

In [ ]:
for row in soup.find('table').find_all('tr')[:10]:
  print(row.text)

You can continue chaining down through as much of the HTML DOM as you'd like!

In [ ]:
(soup
 .find('div', id='content')
 .find('div', id='bodyContent')
 .find('div', id='mw-content-text')
 .find('div', role='note')
).text

### Locating information by position

We just saw that basic facts about Pennsylvania can be found within the first table of this page.  Now let's get more specific.

How can we extract the date Pennsylvania was admitted to the union?

In [ ]:
soup.find('table').find_all('td')[9]

It's the tenth element in this list, but what happens to this code if someone edits the Wikipedia table to include additional information?


Sometimes it is better or necessary to find information by text matching, but be careful -- this needs to be an exact match!

In [ ]:
soup.find(text='Admitted')   #not an exact match

In [ ]:
soup.find(text='Admitted to the Union')

Alternatively, we could use [regular expressions](https://docs.python.org/3/library/re.html).

In [ ]:
import re

admitted_regex = re.compile('Admitted')
soup.find(text=admitted_regex)

This looks like a string, but it's actually a `BeautifulSoup` element.

In [ ]:
admitted = soup.find(text='Admitted to the Union')

type(admitted)

So we can use it to traverse the DOM.  Here, we will find the next element in the tree.

In [ ]:
admitted.next

In [ ]:
admitted.next.text

For some cases it's much easier to find one element and then move up, down, or sideways within the DOM.  `BeautifulSoup` also allows you to look for `.parent`, `.children`, `.next_sibling`, `.previous_sibling`, [etc.](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#navigating-the-tree)

In [ ]:
admitted.parent

**Tip**: Any "plural" attribute such as `children` or `siblings` will return a generator.  Just loop over the result or convert it to a list.

### Exercises

**Exercise 2 - Capital City**  <br>
>Write code to extract the capital of Pennsylvania from the main table without using list positions.

**Exercise 3 - Reference Links**  <br>
> Print out the text of the first three references (at the bottom of the page).  For an added bonus: can you also print all the external links from these three references?

# Data Preparation

Now that we know how to gather information from the web, what do we do with it?

This data can be
- aggregated to look for trends
- visualized to understand patterns
- leveraged with machine learning algorithms





But first we need to 
- convert several strings into numerical or datetime values
- collect and store data from multiple pages (next section)

**Tip**: Most web scraping project rely on multiple pages of information, each of which serving as a data observation.  For this case, we might collect data about Pennsylvania and then collect the same kinds of information for all 50 United States before analyzing or visualizing the data.

### Data processing

#### Date Admitted

In the last section, we collected the date that Pennsylvania was admitted to the union.

In [ ]:
date_admitted_text = admitted.next.text

date_admitted_text

We need Python to recognize this as a date for futher analyses.  First let's narrow down to just the date part of the string.

In [ ]:
date_admitted_list = date_admitted_text.split(' ')[:-1]

date_admitted_list

In [ ]:
date_admitted_str = ' '.join(date_admitted_list)

date_admitted_str

Now we will convert this string into a datetime data type.

In [ ]:
import dateutil.parser

In [ ]:
date_admitted = dateutil.parser.parse(date_admitted_str)

date_admitted

In [ ]:
type(date_admitted)

In [ ]:
date_admitted.year

#### Population and Area

Another quantity that might be useful if we want to compare Pennsylvania to other US states is population.  Let's look for the word "Total" and use the same trick we tried before.





In [ ]:
soup.find(text=re.compile('Total'))

In [ ]:
soup.find(text=re.compile('Total')).next

That's not the population!  Looks like total area is also next to a "Total" label.  Let's save that and come back to it later.

In [ ]:
area_text = soup.find(text=re.compile('Total')).next.text

How might we explicitly look for the population total?

In [ ]:
soup.find(text='Population').parent

In [ ]:
soup.find(text='Population').parent.parent

In [ ]:
soup.find(text='Population').parent.parent.next_sibling

In [ ]:
soup.find(text='Population').parent.parent.next_sibling.find('td')

In [ ]:
population_text = soup.find(text='Population').parent.parent.next_sibling.find('td').text

population_text

Sometimes you need to continuing traversing the DOM until you find the information you need!

Now let's convert that string into an integer.

In [ ]:
population = int(population_text.replace(',', ''))

population

Often it's useful to write functions to help you clean up your data.  Let's do that now so we can reuse these steps.

In [ ]:
def to_date(date_str):
    date_str = re.match('[\w\s,]+', date_str)[0]
    return dateutil.parser.parse(date_str)

def to_int(number_str):
    number_str = re.match('[\d,$]+', number_str)[0]
    number_str = number_str.replace('$', '').replace(',', '')
    return int(number_str)

Now we can use our `to_int` function to clean up the area text we found previously.  This text actually contains special spaces so we will use regular expressions (regex) to capture just the first digits in the `to_int` function.

In [ ]:
area_text

In [ ]:
area = to_int(area_text)

area

### Data storage

Now let's put all the information we have about Pennsylvania together.

In [ ]:
penn_dict = {
    'state': state,
    'date_admitted': date_admitted,
    'population': population,
    'area_sq_mi': area
}

In [ ]:
penn_dict

Once we have this information in dictionary form, we can build a `pandas` dataframe with it and eventually perform further analyses or save it to our computer.

In [ ]:
import pandas as pd

In [ ]:
penn_info = [penn_dict]

In [ ]:
penn_df = pd.DataFrame(penn_info)

penn_df

In [ ]:
penn_df.to_csv('Penn_State_Information.csv')

### Exercises

**Exercise 4 - Median Household Income**  <br>
>Get the median household income for the state of Pennsylvania as a text string and then as an integer.

**Exercise 5 - Median Household Income - Part II**  <br>
> Update `state_df` to include median household income. 

(Hint: One way you can do this: add median household income to `penn_dict` and recreate `state_df`.)

# Pipeline Considerations

Now that we can extract numerical data from this page about Pennsylvania, how would we build out a full analytic or data science project? 

The next step is to systematically retrieve this information from the Wikipedia page of each US state.  First, let's build reusable functions to find the state's
- name
- date admitted
- population
- area
- median household income

Note: all of this info can be found in the table on the right side of the page.

In [ ]:
def get_name(table):
    raw_name = table.find('th').text
    return re.match('[A-z\s]+', raw_name)[0]  

def get_date_admitted(table):
    raw_date = table.find(text='Admitted to the Union').next.text
    return to_date(raw_date)

def get_population(table):
    raw_population = table.find(text='Population')\
                        .parent.parent.next_sibling\
                        .find('td').text
    return to_int(raw_population)

def get_area(table):
    raw_area = table.find(text=re.compile('Total')).next.text
    return to_int(raw_area)

def get_income(table):
    raw_income = table.find(text='Median household income').next.next.text
    return to_int(raw_income)

These functions will extract information from any Wikipedia state table we pass into them.  For example, let's try parsing the page for New York.

In [ ]:
ny_url = 'https://en.wikipedia.org/wiki/New_York_(state)'

ny_page = requests.get(ny_url).text

ny_soup = bs(ny_page)

In [ ]:
ny_table = ny_soup.find('table')

In [ ]:
get_name(ny_table)

In [ ]:
get_population(ny_table)

Let's also make a function to gather all five values from a given state Wiki page and return the information as a dictionary.

In [ ]:
def parse_url(url):
    page = requests.get(url).text
    return bs(page)


def get_state_info(state_url):
  
    #Use parse page and grab main table
    state_soup = parse_url(state_url)
    state_table = state_soup.find('table')

    state_info = {}

    #Grab info with pre-defined functions
    state_info['state'] = get_name(state_table)
    state_info['date_admitted'] = get_date_admitted(state_table)
    state_info['population'] = get_population(state_table)
    state_info['area'] = get_area(state_table)
    state_info['median_household_income'] = get_income(state_table)

    return state_info

In [ ]:
ny_info = get_state_info(ny_url)

ny_info

### Lists of links

The next step in our process will require us to use our `get_state_info()` function on the URLs of each of the 50 US states.  But how do we know which URLs to visit?  We might be able to guess that the page for Rhode Island is https://en.wikipedia.org/wiki/Rhode_Island but not all pages follow this convention.

In [ ]:
ny_url

Instead of guessing, let's first gather these links from this "[List of States and Territories of the United States](https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States)" article. 

Click on this link and inspect the page to develop a plan for doing this.

It looks like each of the states are listed in the second table of the page.  Each state name and link is contained within table header tags (`th`) and have the additional property of `scope`="row".

In [ ]:
list_url = 'https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States'
list_page = requests.get(list_url).text
list_soup = bs(list_page)

In [ ]:
state_rows = list_soup.find_all('table')[1].find_all('th', scope='row')

state_rows[:5]

Now we just need to extract the links from the `a` tags.

In [ ]:
state_rows[0].find('a')

In [ ]:
state_rows[0].find('a')['href']

In [ ]:
state_links = [row.find('a')['href'] for row in state_rows]

state_links[:5]

Each of these links point to a place within Wikipedia, but if we want to link to the full URLs, we have to append 'https://en.wikipedia.org' to each of them.

In [ ]:
base_url = 'https://en.wikipedia.org'

state_urls = [base_url + link for link in state_links]

state_urls[:5]

In [ ]:
state_urls[-5:]

In [ ]:
len(state_urls)

### Handling missing values

We will eventually be cycling through these state links to collect and store information about every state.  But what happens when certain information is unavailable?  That is, what if the Georgia page is missing area information or the median household income isn't listed for Nevada?

We can make our code more robust by including instructions for handling missing information.  One way to do this is to include `try`/`except` statements.

If you haven't seen them before, `try`/`except` pairs are used to let Python know how to handle errors.

In [ ]:
def square(x):
    return x*x

In [ ]:
square('hi')  #This returns an error

In [ ]:
def square_robust(x):
    try:
        return x*x
    except TypeError:
        return "No can do!"

In [ ]:
square_robust('hi')

In the case of web scraping, we may be scraping information from many, many pages.  If any of the information we want can't be found, we usually don't want Python to exit the program with an error.  We typically prefer that Python continue the scraping but just fill in that particular piece of information with a missing value like `None`.

```
def my_scraper(page):
  try:
    perform some parsing
    return my_scraped_value
  except:
    return None
```

Let's update the collection of our state info to be robust to handling missing values.

In [ ]:
def get_state_info_robust(state_url):
  
    #If we can't find a main table, 
    #  print out the url and exit function
    try:
        state_soup = parse_url(state_url)
        state_table = state_soup.find('table')
    except:
        print(f"Cannot parse table: {state_url}")
        return None

    state_info = {}

    #Grab info with pre-defined functions
    #  If any value can't be found, just fill value with None
    values = ['state', 'date_admitted', 'population', 
            'area_sq_mi', 'median_household_income']
    functions = [get_name, get_date_admitted, get_population,
               get_area, get_income]

    for val, func in zip(values, functions):
        try:
            state_info[val] = func(state_table)
        except:
            state_info[val] = None

    return state_info

In [ ]:
ny_dict = get_state_info_robust(ny_url)

ny_dict

In [ ]:
get_state_info_robust('https://en.wikipedia.org/wiki/Python_Conference')

In [ ]:
get_state_info_robust('https://notawebsiteatleastihopenot.net')

### Adding pauses

We have just one final consideration before we cycle through the state links to scrape information.  Web scraping at a fast rate--that is, many pages per second--is frowned upon by many websites, Wikipedia included.  We will add in artificial pauses so we don't overwhelm the Wikipedia server.

In [ ]:
import time

In [ ]:
time.sleep(2)

In [ ]:
a = 5

print(f"Pausing for {a} seconds")
time.sleep(a)

b = a + 1
print(f"b equals {b}")

To responsibly scrape websites, you should know what the site's rate limit is and respect it!  Most sites list their rate limit for web scraping in their `robots.txt` file.  More on this later. 

Wikipedia requests [at least a one second pause per page request](https://en.wikipedia.org/wiki/Wikipedia:Database_download#Please_do_not_use_a_web_crawler).  We will pause 1 second between each page scrape, so we will only **collect information for 10 US states** for now.

**WARNING!!** Not respecting site's limits can get your IP address blocked from the site.  Don't get yourself blocked from Wikipedia!

### Data storage revisited

We now have a function to extract information for each state as a dictionary.  We can convert this information into a `pandas` dataframe and store it to an Excel or .csv file if we pass in a list of dictionaries, all with the same keys.

In [ ]:
[penn_dict, ny_dict]

In [ ]:
pd.DataFrame([penn_dict, ny_dict])

Now let's build out our full pipeline:

1. Gather a list of links to each state. (DONE)
2. For each state link, gather state information as a dictionary.
3. Append each state dictionary to a list.
4. Convert list of dictionaries to dataframe.
5. Save dataframe as a .csv or an Excel file.

In [ ]:
state_info_list = []

for link in state_urls[:10]:

  #Step 2.
  state_info = get_state_info_robust(link)

  #Step 3.
  if state_info:
    state_info_list.append(state_info)

  #Be sure to pause
  time.sleep(1)

In [ ]:
state_info_list

In [ ]:
#Step 4.
state_data = pd.DataFrame(state_info_list)

state_data

In [ ]:
#Step 5.
state_data.to_csv('state_data.csv', index=False)

In [ ]:
files.download('state_data.csv')

Now that you have the state information in .csv format, you can analyze it with any tool you know how to use:
- Excel
- visualization tools like Tableau
- additional Python code

Note: If you DID gather information for all 50 states, you would find one missing value: Kansas's date admitted.  After visiting/inspecting the Wiki page for [Kansas](https://en.wikipedia.org/wiki/Kansas), do you see why? How could you fix the data extraction function to account for this issue?

### Systematically named pages

For many web scraping projects, you will begin by collecting links from a links page.  Other times you might be able to devise a pattern in the URLs that you can exploit.

**Example: Billboard Year-End Hot 100**

The _Billboard_ Hot 100 chart is well known for tracking the success of music singles within the US.  At the end of each year, Billboard compiles a list of the top 100 performing songs throughout the year based on the information from Hot 100 charts.  Wikipedia displays this information as an article here: https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2019

How might we compile a list of the most popular song for each year since 2010?

In [ ]:
top_hits = []

for year in range(2010, 2020):

    #Build URL for each year
    base_url = 'https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_'
    url = base_url + str(year)
    print(url)

    page = requests.get(url).text
    soup = bs(page)

    #Grab top hit text and link
    top_hit = soup.find('table', class_='wikitable').find('td')
    top_hit_text = top_hit.text
    try:
        top_hit_link = top_hit.find('a')['href']
    except:
        top_hit_link = None

    #Store results as list of tuples
    top_hits.append((year, top_hit_text, top_hit_link))

    #Be sure to pause
    time.sleep(1)

In [ ]:
top_hits

# Mini Projects

To practice putting together all of the skills you have learned today, you will be working on your very own mini project.  

### Tips
Some project advise before getting started:
- **Start small and scale up.**  Make sure your code is working on one page before you try to request information from a ton of links.
- **Think through data storage before scaling up.** How will you store the information so you can perform analyses on the data you collect?
- **Safeguard against missing values.** It is so annoying when a scraping loop breaks on the last link and all other information is lost...
- **Pause for 1+ seconds between requests.** Let's try to not get banned from Wikipedia!

With those tips in mind, let's get started on a mini project.  Further directions and project ideas can be found here: https://github.com/kimfetti/Conferences/blob/master/PyCon_2020/mini_project_ideas.md

---

# Solutions

 **Class Exercise - Disambiguation link**  <br>
>Let's try to extract this disambiguation link by inspecting the source code and then using `BeautifulSoup`.  

In [ ]:
soup.find(class_='mw-disambig')

In [ ]:
soup.find(class_='mw-disambig')['href']

 **Exercise 1 - Longitute, Latitude**  <br>
>How would you retrieve Pennsylvania's longitude and latitude coordinates from this page?

<center>
<img src="https://drive.google.com/uc?export=view&id=1N43WC3jwFpV80L7-iLnEZi8aCQZtWcd3"  alt="coordinates" width="600"/>
</center>

_Hint: Right click the coordinates directly and then select inspect._

In [ ]:
soup.find(class_='geo-dec').text

 **Exercise 2 - Capital City**  <br>
>Write code to extract the capital of Pennsylvania from the main table without using list positions.

In [ ]:
soup.find(text='Capital')

In [ ]:
soup.find(text='Capital').next

In [ ]:
soup.find(text='Capital').next.text

 **Exercise 3 - Reference Links**  <br>
> Print out the text of the first three references (at the bottom of the page).  For an added bonus: can you also print all the external links from these three references?

In [ ]:
references = soup.find(class_='reflist').find_all('li')[:3]

for ref in references:
    print(ref.text)

In [ ]:
for ref in references:
    for link in ref.find_all('a', class_='external'):
        print(link['href'])

**Exercise 4 - Median Household Income - Part I**  <br>
>Get the median household income for the state of Pennsylvania as a text string and then as an integer.

In [ ]:
mhi_text = soup.find(text='Median household income').next.next.text

mhi_text

In [ ]:
mhi_int = to_int(mhi_text)

mhi_int

 **Exercise 5 - Median Household Income - Part II**  <br>
> Update `state_df` to include median household income. 

(Hint: One you can do this: add median household income to `penn_dict` and recreate `state_df`.)

In [ ]:
penn_dict['median_household_income'] = mhi_int

In [ ]:
penn_df = pd.DataFrame([penn_dict])

penn_df

In [ ]:
# #Alternatively...

# state_df['median_household_income'] = [mhi_int]